In [1]:
%load_ext autoreload
%autoreload 2

from spot.utils import proj_root, os
os.chdir(proj_root())

In [2]:
import requests
from spot.utils import proj_root, read_file

bad_code = """
def parse_qualified_name(tree: ast.Attribute):
    segs = []
    while isinstance(tree, ast.Attribute):
        segs.append(tree.attr)
        tree = tree.value  # type: ignore
    assert isinstance(tree, ast.Name)
    segs.append(tree.id)
    return tuple(reversed(segs))
"""

r = requests.post("https://type4py.com/api/predict?tc=0", bad_code)
r.json()


{'error': None,
 'response': {'classes': [],
  'funcs': [{'docstring': {'func': None, 'long_descr': None, 'ret': None},
    'fn_lc': [[2, 0], [9, 32]],
    'fn_var_ln': {'segs': [[3, 4], [3, 8]], 'tree': [[6, 8], [6, 12]]},
    'fn_var_occur': {'segs': [['segs', 'append', 'tree', 'attr'],
      ['segs', 'append', 'tree', 'id'],
      ['tuple', 'reversed', 'segs']],
     'tree': [['isinstance', 'tree', 'ast', 'Attribute'],
      ['segs', 'append', 'tree', 'attr'],
      ['tree', 'tree', 'value'],
      ['isinstance', 'tree', 'ast', 'Name'],
      ['segs', 'append', 'tree', 'id']]},
    'name': 'parse_qualified_name',
    'params': {'tree': 'ast.Attribute'},
    'params_descr': {'tree': ''},
    'params_occur': {'tree': [['isinstance', 'tree', 'ast', 'Attribute'],
      ['segs', 'append', 'tree', 'attr'],
      ['tree', 'tree', 'value'],
      ['isinstance', 'tree', 'ast', 'Name'],
      ['segs', 'append', 'tree', 'id']]},
    'params_p': {'args': [], 'kwargs': [], 'tree': []},
    'q_na

In [3]:
from spot.experiments.type4py import Type4PyResponseParser
from spot.static_analysis import FunctionSignature

pred_signatures = Type4PyResponseParser("spot.model").parse(r.json())
for path, sig in pred_signatures.items():
    if isinstance(sig, FunctionSignature):
        print(path)
        print("\t" + str(sig))


spot.model/parse_qualified_name
	FuncSig(() -> int)


In [8]:
from spot.static_analysis import PythonProject
from spot.utils import *
from spot.model import ModelWrapper
from spot.visualization import pretty_print_dict, assert_eq
from spot.experiments.type4py import eval_type4py_on_projects
from spot.function_dataset import data_project_from_dir


dataset_name = "ManyTypes4Py"
# dataset_name = "InferTypes4Py"

# test_projects = [PythonProject.from_root(proj_root(), ignore_dirs={".venv", "data"})]

repos_dir = get_dataset_dir(dataset_name) / "repos" / "test"
test_repo_paths = [f for f in repos_dir.iterdir() if f.is_dir()]
test_projects = pmap(
    data_project_from_dir,
    test_repo_paths,
    desc="Loading test projects",
)
cache = PickleCache(Path(f"caches/run_type4py"))

eval_r = cache.cached(f"{dataset_name}.pkl", lambda: eval_type4py_on_projects(test_projects, max_workers=4))


Loading test projects: 100%|██████████| 50/50 [00:23<00:00,  2.15it/s]


In [9]:
from spot.static_analysis import SignatureErrorAnalysis, AccuracyMetric
from spot.experiments.typet5 import accs_as_table_row


common_names = ModelWrapper.load_common_type_names(
    get_model_dir() / "model-v7--TrainingConfig(drop_env_types=False)"
)
metrics = AccuracyMetric.default_metrics(common_type_names=common_names)
# acc_metric = AccuracyMetric(common_type_names=ubiq_names)

accs = {
    m.name: SignatureErrorAnalysis(
        eval_r.pred_maps,
        eval_r.label_maps,
        m,
        error_on_mismatched_signature=False,
    ).accuracies
    for m in metrics
}

accs_as_table_row(accs)
pretty_print_dict(accs)

Accuracies on all types:
header:  ['full.all', 'calibrated.simple', 'calibrated.complex', 'calibrated.all', 'base.all']
34.27 & 35.87 & 19.68 & 34.52 & 35.61
Accuracies on common types:
header:  ['full.all', 'calibrated.simple', 'calibrated.complex', 'calibrated.all', 'base.all']
52.54 & 51.91 & 32.14 & 50.34 & 47.51
full_acc:
   full_acc: 34.27% (count=15.2k)
   full_acc_by_cat:
      FuncArg: 28.54% (count=7.5k)
      FuncReturn: 45.84% (count=5.7k)
      ClassAtribute: 22.67% (count=1.8k)
      GlobalVar: 13.33% (count=105)
   full_acc_by_simple:
      complex: 14.02% (count=1.7k)
      simple: 36.92% (count=13.4k)
   full_acc_label_size: 1.4294
   full_acc_pred_size: 1.2304
   full_acc_ignored_labels: 0
   n_skipped_types: 271
   n_missing_types: 323
full_acc_common:
   full_acc_common: 52.54% (count=9.8k)
   full_acc_common_by_cat:
      FuncArg: 50.36% (count=5.0k)
      FuncReturn: 55.01% (count=3.8k)
      ClassAtribute: 54.56% (count=1.0k)
      GlobalVar: 45.24% (count=84)
  